In [4]:
import requests 
from bs4 import BeautifulSoup
from urllib.parse import urlencode

In [5]:
import pandas as pd
import numpy as np

In [6]:
## 위도 경도 추출
def get_addr_info(keyword=""):
    url = "https://apis.zigbang.com/search/all?q={}&type=oneroom".format(keyword)
    response = requests.get(url)
    json_obj = response.json()
    return json_obj["items"][0]["lat"], json_obj["items"][0]["lng"]
    
# test code    
lat, lng = get_addr_info("성수동")
lat, lng

(37.541725158691406, 127.04335021972656)

In [7]:
def get_ids(lat, lng):
    
    url = "https://api.zigbang.com/v3/items2?\
lat_south={}&lat_north={}&lng_west={}&lng_east={}\
&room=[01,02,03,04,05]".format(lat - 0.01, lat + 0.01, lng - 0.01, lng + 0.01)
    
    params = {
        "lat_south": lat - 0.01,
        "lat_north": lat + 0.01,
        "lng_west": lng - 0.01,
        "lng_east": lng + 0.01,
        "room": "[01,02,03,04,05]",
    }
    params_str = urlencode(params)
    url = "https://api.zigbang.com/v3/items2?" + params_str
    response = requests.get(url)
    json_obj = response.json()
    items = json_obj["list_items"]
    return [item["simple_item"]["item_id"] for item in items]

# test code
ids = get_ids(lat, lng)
len(ids), ids[:5]

(76, [17468916, 17469110, 17279197, 17773125, 17470120])

In [8]:
from pandas.io.json import json_normalize
def get_items(ids):
    url = "https://api.zigbang.com/v3/items?detail=true&item_ids={}"\
.format(str(ids).replace(" ",""))
    response = requests.get(url)
    json_obj = response.json()
    items = json_obj["items"]
    datas = [item["item"] for item in items]
    result_df = json_normalize(datas)
    filter_colums = ["_floor", "address1", "address2", 
                     "user_email", "rent", "size", "deposit", "options"]
    return result_df[filter_colums]
    
# test code    
result_df = get_items(ids)    
result_df.iloc[:9]

,_floor,address1,address2,user_email,rent,size,deposit,options
0,2층,서울시 성동구 성수동1가,13-338,25highlight25@gmail.com,45,6.5,500,"에어컨,냉장고,세탁기,가스레인지,싱크대"
1,2층,서울시 성동구 성수동1가,13-338,15highlight15@gmail.com,45,6.5,500,"에어컨,냉장고,가스레인지,싱크대"
2,2층,서울시 성동구 성수동1가,656-5,70ktcho@hanmail.net,0,8.0,5500,"에어컨,냉장고,세탁기"
3,2층,서울시 성동구 성수동1가,656-348,sang2205@naver.com,60,8.0,2000,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"
4,2층,서울시 성동구 성수동1가,13-338,05highlight05@gmail.com,45,6.5,500,"에어컨,냉장고,세탁기,가스레인지,싱크대"
5,2층,서울시 성동구 성수동1가,31-35,soek0369@naver.com,60,9.0,1000,"에어컨,냉장고,세탁기,가스레인지,옷장,신발장,싱크대"
6,7층,서울시 성동구 성수동1가,13-55,lsj6384@naver.com,55,8.0,1000,"에어컨,냉장고,세탁기,가스레인지,옷장,신발장,싱크대"
7,2층,서울시 성동구 성수동1가,13-338,06highlight06@gmail.com,45,6.5,500,"에어컨,냉장고,세탁기,가스레인지,싱크대"
8,3층,서울시 성동구 성수동1가,13-109,phsoon23@naver.com,55,7.0,1000,"에어컨,냉장고,세탁기,전자레인지,책상,옷장,싱크대"


In [9]:
def main(addr=""):
    lat, lng = get_addr_info(addr)
    ids = get_ids(lat, lng)
#     print(len(ids))
    return get_items(ids)    

In [10]:
# test code
aa = main('부평구') 
aa

,_floor,address1,address2,user_email,rent,size,deposit,options
0,2층,인천시 부평구 부평동,132-4,jjyjsb@naver.com,45,10.0,100,"에어컨,냉장고,세탁기,인덕션,책장,옷장,신발장,싱크대"
1,1층,인천시 부평구 부평동,465-14,sahera@naver.com,28,5.5,30,"에어컨,냉장고,세탁기,가스레인지,전자레인지,신발장,싱크대"
2,4층,인천시 부평구 부평동,374-18,oopunkoo@naver.com,20,8.0,2500,"에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,옷장,신발장,싱크대"
3,2층,인천시 부평구 부평동,10-485,ykpjjang@hanmail.net,45,7.0,50,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"
4,5층,인천시 부평구 부평동,529-32,qorhks3@naver.com,40,8.0,1000,"에어컨,냉장고,인덕션,책상,책장,옷장,신발장,싱크대"
5,3층,인천시 부평구 부평동,372-57,sahera@naver.com,33,5.0,50,"에어컨,냉장고,세탁기,가스레인지,옷장,신발장,싱크대"
6,6층,인천시 부평구 부평동,400-25,sahera@naver.com,35,7.0,500,"에어컨,냉장고,세탁기,인덕션,책장,옷장,신발장,싱크대"
7,15층,인천시 부평구 부평동,372-57,ahrami_@naver.com,50,7.5,60,"냉장고,세탁기,가스레인지,책상,옷장,신발장,싱크대"
8,4층,인천시 부평구 부평동,465-27,kkk7433@hanmail.net,33,9.0,500,"에어컨,냉장고,세탁기,가스레인지,침대,신발장,싱크대"
9,5층,인천시 부평구 부평동,415-26,ahrami_@naver.com,45,8.0,300,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대"


In [14]:
## json 구조 접근법
json_data # 중첩 json 구조로 되어있음. 'name','version','url','data' 의 key값들 중 'data'에 접근해야함

# json_data['data'] # list의 안의 json 구조로 되어있음. 따라서 우선 리스트로 접근하기

# json_data['data'][0] # 다시 json구조 , 다시 key값으로 해야함

# json_data['data'][0]['서울특별시'] # 결국 서울특별시에 해당하는 구들이 list형태로 생김

# res = [] # 구 list를 담기위한 list
# res.append(json_data['data'][0]['서울특별시']) # res에 리스트 append
# res.append(json_data['data'][1]['부산광역시'])
# res.append(json_data['data'][2]['인천광역시'])

# sum_res = sum(res,[]) #리스트안의 리스트들 구조로되어있기 때문에 접근을 더 쉽게 하기위해 list를 일원화 시킴 [[]] => [] 

{'name': 'korea-administrative-district',
 'version': '20160125',
 'url': 'https://github.com/cosmosfarm/korea-administrative-district',
 'data': [{'서울특별시': ['종로구',
    '중구',
    '용산구',
    '성동구',
    '광진구',
    '동대문구',
    '중랑구',
    '성북구',
    '강북구',
    '도봉구',
    '노원구',
    '은평구',
    '서대문구',
    '마포구',
    '양천구',
    '강서구',
    '구로구',
    '금천구',
    '영등포구',
    '동작구',
    '관악구',
    '서초구',
    '강남구',
    '송파구',
    '강동구']},
  {'부산광역시': ['중구',
    '서구',
    '동구',
    '영도구',
    '부산진구',
    '동래구',
    '남구',
    '북구',
    '강서구',
    '해운대구',
    '사하구',
    '금정구',
    '연제구',
    '수영구',
    '사상구',
    '기장군']},
  {'인천광역시': ['중구',
    '동구',
    '남구',
    '연수구',
    '남동구',
    '부평구',
    '계양구',
    '서구',
    '강화군',
    '옹진군']},
  {'대구광역시': ['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군']},
  {'광주광역시': ['동구', '서구', '남구', '북구', '광산구']},
  {'대전광역시': ['동구', '중구', '서구', '유성구', '대덕구']},
  {'울산광역시': ['중구', '남구', '동구', '북구', '울주군']},
  {'세종특별자치시': []},
  {'경기도': ['가평군',
    '고양시',
    '과천시',
  

In [15]:
import json
## encoding utf-8
with open('korea-administrative-district.json', encoding='utf-8') as jsonfile: # korea-administrative-district.json 행정구역 json
    json_data = json.load(jsonfile)
## 시 추출    
parsing_list = []
for i in range(0,3):
    parsing_list.append(list(json_data['data'][i].keys())) 

parsing_data = []
# parsing_data.append(list(json_data['data'][0]['서울특별시']))
parsing_data.append(list(json_data['data'][1]['부산광역시']))
# parsing_data.append(list(json_data['data'][2]['인천광역시']))
    
## 구/군 추출
parsing_data = sum(parsing_data,[])

In [16]:
json_data

{'name': 'korea-administrative-district',
 'version': '20160125',
 'url': 'https://github.com/cosmosfarm/korea-administrative-district',
 'data': [{'서울특별시': ['종로구',
    '중구',
    '용산구',
    '성동구',
    '광진구',
    '동대문구',
    '중랑구',
    '성북구',
    '강북구',
    '도봉구',
    '노원구',
    '은평구',
    '서대문구',
    '마포구',
    '양천구',
    '강서구',
    '구로구',
    '금천구',
    '영등포구',
    '동작구',
    '관악구',
    '서초구',
    '강남구',
    '송파구',
    '강동구']},
  {'부산광역시': ['중구',
    '서구',
    '동구',
    '영도구',
    '부산진구',
    '동래구',
    '남구',
    '북구',
    '강서구',
    '해운대구',
    '사하구',
    '금정구',
    '연제구',
    '수영구',
    '사상구',
    '기장군']},
  {'인천광역시': ['중구',
    '동구',
    '남구',
    '연수구',
    '남동구',
    '부평구',
    '계양구',
    '서구',
    '강화군',
    '옹진군']},
  {'대구광역시': ['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군']},
  {'광주광역시': ['동구', '서구', '남구', '북구', '광산구']},
  {'대전광역시': ['동구', '중구', '서구', '유성구', '대덕구']},
  {'울산광역시': ['중구', '남구', '동구', '북구', '울주군']},
  {'세종특별자치시': []},
  {'경기도': ['가평군',
    '고양시',
    '과천시',
  

In [20]:

import cx_Oracle # 파이썬 oracle 연동 모듈 

con = cx_Oracle.connect("KIHYUK/3927@192.168.0.216:1521/orcl") # 연결 설정

for data in parsing_data:
    
    df = main(data)
    rows = list(df.itertuples(index=False,name=None)) # dataframe을 한줄씪 tuple로 만들어서 list에 넣음
#   tuple 형태로 넣어줘야함
#   [(0,a,f)] => 0,a,f 들어감( 컬럼에맞게)
    cursor = con.cursor()
    cursor.executemany("INSERT INTO HOUSE_TE(FLOOR,ADDRESS1,ADDRESS2,USER_EMAIL,RENT,SIZES,DEPOSIT,OPTIONS) VALUES(:1,:2,:3,:4,:5,:6,:7,:8)",rows)
    con.commit()

con.close()

     _floor      address1 address2               user_email  rent  size  \
0        2층  서울시 중구 장충동2가  193-144     msjeon1662@naver.com     0   4.8   
1        3층  서울시 중구 장충동2가  193-133     msjeon1662@naver.com    45   5.7   
2        3층  서울시 중구 장충동2가  193-265       na5888@hanmail.net    60   8.0   
3        4층  서울시 중구 장충동2가   193-84     msjeon1662@naver.com    47   6.4   
4        2층   서울시 종로구 연지동    238-3      woori0999@naver.com     0   6.3   
5        2층  서울시 중구 장충동2가   187-28    happyhappy70@daum.net    65   6.0   
6        4층  서울시 중구 장충동2가   193-86         ko00477@daum.net    45   4.0   
7        5층   서울시 종로구 연지동      184        macho16@naver.com    65   6.0   
8        2층    서울시 중구 묵정동    18-27      judongc@hanmail.net    65  10.0   
9        2층  서울시 중구 장충동2가    188-8    happyhappy70@daum.net    45   4.5   
10       3층  서울시 중구 장충동2가  193-483     msjeon1662@naver.com    50   5.6   
11       2층   서울시 종로구 연지동    238-3      woori0999@naver.com     0   7.0   
12       2층  서울시 중구 장충동1가

    _floor    address1 address2            user_email  rent  size  deposit  \
0       2층  광주시 서구 농성동   345-86   skytopgun@naver.com     0  13.0     6000   
1       4층  광주시 서구 농성동   641-18     gj1room@naver.com    37  15.0      200   
2       2층  광주시 서구 농성동    640-1   water8591@naver.com    25  11.0      100   
3       2층  광주시 서구 농성동    644-8     dps8888@naver.com    30  10.0      200   
4       3층  광주시 서구 농성동   345-86     jeh7184@naver.com    30  12.0      200   
5       3층  광주시 서구 농성동   646-33     dps8888@naver.com    23  10.0      200   
6       3층  광주시 서구 농성동    392-9    iminto82@naver.com    28  10.0      200   
7       4층  광주시 남구 월산동    962-4     gj1room@naver.com    40  15.0      300   
8       2층  광주시 서구 농성동   645-25      kjgolf@naver.com    24   9.0      100   
9       2층  광주시 서구 농성동   345-86     jeh7184@naver.com    30  11.0      200   
10      3층  광주시 서구 농성동   345-86     dps8888@naver.com    30  13.0      200   
11      2층  광주시 남구 월산동    950-8   skytopgun@naver.com    30  10.

   _floor    address1 address2            user_email  rent  size  deposit  \
0      2층  대전시 동구 가오동      440    fivesunya@daum.net    30  10.0      300   
1      2층  대전시 동구 신흥동     20-1    12ls12@hanmail.net     0   6.0     4000   
2      3층  대전시 동구 가오동      573    fivesunya@daum.net     0   8.0     4000   
3      2층  대전시 동구 판암동    550-3      ghkd991@nate.com     0  10.0     5500   
4      2층  대전시 동구 가오동   237-11    fivesunya@daum.net    30   9.0      300   
5      4층  대전시 동구 판암동    550-3   shimsrich@naver.com     0  10.0     6000   
6      3층  대전시 동구 가오동      439    fivesunya@daum.net    23   7.0      100   
7      1층  대전시 동구 가오동      575    fivesunya@daum.net    38  15.0      300   
8      2층  대전시 동구 가오동      569    fivesunya@daum.net    30   9.0      500   
9      3층  대전시 동구 가오동      582    fivesunya@daum.net    30  10.0      300   
10     4층  대전시 동구 판암동    470-7       y2393@naver.com     0   5.0     4000   
11     2층  대전시 동구 가오동      531    fivesunya@daum.net    35  10.0      500   

    _floor     address1 address2               user_email  rent  size  \
0       6층  부산시 동래구 온천동   480-22       suni3788@naver.com    46  10.0   
1       5층  부산시 동래구 온천동  1441-41       suni3788@naver.com    40   7.0   
2       3층  부산시 동래구 온천동  1442-13      wprnlghks@naver.com    42   8.0   
3       6층  부산시 동래구 온천동   478-16     yebebe8585@naver.com     0  11.0   
4       4층  부산시 동래구 온천동  1441-44        nsy5944@naver.com    53  12.0   
5       2층  부산시 동래구 온천동  1441-49       suni3788@naver.com    40   8.0   
6       4층  부산시 동래구 온천동   1453-5       suni3788@naver.com    40   9.0   
7       5층  부산시 동래구 수안동      2-9       jimbab72@naver.com    30  13.0   
8       9층  부산시 동래구 온천동   480-23     yebebe8585@naver.com     0   8.7   
9      11층  부산시 동래구 온천동   1442-4       suni3788@naver.com    35   9.0   
10      4층  부산시 동래구 온천동   1453-5       suni3788@naver.com    55   9.0   
11      9층  부산시 동래구 온천동   478-16       jimbab72@naver.com    30  13.0   
12      9층  부산시 동래구 온천동   480-21        nsy5944@nav

    _floor    address1 address2               user_email  rent  size  deposit  \
0       2층  광주시 북구 문흥동    733-3  jikbang6061@hanmail.net    30   9.0      200   
1       3층  광주시 북구 신안동   478-71  jikbang6061@hanmail.net    31   8.0      200   
2       1층  광주시 북구 중흥동   260-28  jikbang6061@hanmail.net    22   9.0      100   
3       4층  광주시 북구 우산동   554-62        popo220@naver.com    24   9.0      200   
4       3층  광주시 북구 용봉동   125-34        popo220@naver.com    23  11.0      100   
5       2층  광주시 북구 용봉동   125-34        popo220@naver.com    25  10.0      100   
6       3층  광주시 북구 문흥동   739-22        popo220@naver.com    29   9.0      200   
7       3층  광주시 북구 중흥동    337-4  jikbang6061@hanmail.net    25   9.0      100   
8       2층  광주시 북구 신안동   478-50        popo220@naver.com    27   9.0      100   
9       2층  광주시 북구 용봉동    148-3        popo220@naver.com    25   9.0      200   
10      2층  광주시 북구 용봉동   1227-6        popo220@naver.com    28   9.0      100   
11      1층  광주시 북구 용봉동  1205

...

     _floor     address1 address2               user_email  rent  size  \
0        3층  부산시 사하구 하단동   618-37      tjdwns168@naver.com    25   9.0   
1        2층  부산시 사하구 괴정동   568-11    gjehdus0701@naver.com    30  14.0   
2        3층  부산시 사하구 하단동   492-21        ybb0509@naver.com    22  13.0   
3        8층  부산시 사하구 하단동   618-15    wimsaa30111@naver.com    32  10.0   
4        5층  부산시 사하구 하단동   497-22        sbb1106@naver.com     0  13.0   
5        5층  부산시 사하구 괴정동   581-14      tjdwns168@naver.com     0  10.0   
6        4층  부산시 사하구 당리동   317-68     choigilhan@naver.com    25  10.0   
7        1층  부산시 사하구 하단동   507-18        ybb0509@naver.com    25  13.0   
8        1층  부산시 사하구 당리동    90-23    gjehdus0701@naver.com    29  14.0   
9        1층  부산시 사하구 당리동    90-25    gjehdus0701@naver.com    29  14.0   
10       4층  부산시 사하구 하단동   534-14        ybb0509@naver.com    35  10.0   
11       5층  부산시 사하구 하단동    701-9  seungwoozzang@naver.com    30  12.0   
12       1층  부산시 사하구 하단동    507-6     

   _floor     address1 address2              user_email  rent  size  deposit  \
0      2층  부산시 금정구 구서동   487-17     sonygod80@naver.com    31  12.0      200   
1      2층  부산시 금정구 장전동   263-16      Jib9haja@naver.com    30   8.0      200   
2      2층  부산시 금정구 장전동    136-5  againlove521@naver.com    28   6.0      300   
3      5층  부산시 금정구 장전동   154-17       nsy5944@naver.com    37   6.0      500   
4      4층  부산시 금정구 장전동    386-2   nicky0076@hanmail.net    35   7.0      300   
5      3층  부산시 금정구 구서동   486-47     sonygod80@naver.com    15  11.0      100   
6      1층  부산시 금정구 장전동   366-24  againlove521@naver.com    25   8.0      200   
7      1층  부산시 금정구 장전동     47-6      Jib9haja@naver.com    25   7.0      100   
8      1층  부산시 금정구 장전동   366-24  againlove521@naver.com    26   8.0      100   
9      1층  부산시 금정구 장전동   152-22  againlove521@naver.com    27   7.0      200   
10     3층  부산시 금정구 장전동    192-1      Jib9haja@naver.com    28   8.0      200   
11     4층  부산시 금정구 장전동    227-4      Jib

    _floor     address1 address2              user_email  rent  size  deposit  \
0       3층   부산시 남구 대연동    30-24      75458450@naver.com    38   7.3      400   
1       4층   부산시 남구 대연동    43-26      wona0804@naver.com    60  11.3     2000   
2       2층   부산시 남구 대연동     31-8    ejskdms123@naver.com    34  13.0     3000   
3       5층   부산시 남구 대연동    30-24     hyeok7385@naver.com    37  12.0      500   
4       4층   부산시 남구 대연동     34-9    wjdahrrud2@naver.com    27   7.0      100   
5       6층   부산시 남구 대연동    42-14     hyeok7385@naver.com    35  12.0      300   
6       5층   부산시 남구 대연동    30-17     hyeok7385@naver.com     0   8.0     3700   
7       6층   부산시 남구 대연동    39-31      75458450@naver.com    32   6.0      500   
8       8층   부산시 남구 대연동    30-21     hyeok7385@naver.com    45  13.0     1000   
9       5층   부산시 남구 대연동    30-17     hyeok7385@naver.com    10   8.0     2000   
10     10층  부산시 수영구 남천동    69-10   cjswjdcjf53@naver.com    33   7.0     1000   
11      9층   부산시 남구 대연동    3

     _floor     address1 address2              user_email  rent  size  \
0        3층  부산시 사상구 괘법동   552-35      1997geun@naver.com    42  10.0   
1        3층  부산시 사상구 괘법동    582-4      1997geun@naver.com    23   9.0   
2        2층  부산시 사상구 감전동   106-21        yo1803@naver.com    30  11.0   
3        3층  부산시 사상구 감전동   122-23      1997geun@naver.com    31  13.0   
4        5층  부산시 사상구 괘법동    550-7      1997geun@naver.com    33  10.0   
5        1층  부산시 사상구 괘법동    552-8        yo1803@naver.com     0   9.0   
6        2층  부산시 사상구 괘법동   550-15        yo1803@naver.com    35  11.0   
7        2층  부산시 사상구 괘법동   557-16      1997geun@naver.com    27   8.0   
8        4층  부산시 사상구 감전동    214-2        yo1803@naver.com    35  11.0   
9        5층  부산시 사상구 괘법동    555-4        yo1803@naver.com    48  13.0   
10       2층  부산시 사상구 괘법동   552-32        yo1803@naver.com     0   9.0   
11       3층  부산시 사상구 괘법동   551-31      1997geun@naver.com     0   8.0   
12       1층  부산시 사상구 괘법동   557-10        yo1803@nav

   _floor         address1 address2            user_email  rent  size  \
0      5층  부산시 기장군 기장읍 청강리    90-38   jupiterkw@naver.com    30  10.8   
1      2층   부산시 기장군 기장읍 교리    326-3  ejskdms123@naver.com     0  11.0   
2      3층  부산시 기장군 기장읍 동부리   434-11   jupiterkw@naver.com    35  12.0   
3      3층   부산시 기장군 기장읍 교리    347-1  ejskdms123@naver.com    28  11.0   
4      3층  부산시 기장군 기장읍 동부리    376-1   jupiterkw@naver.com    28   8.0   
5      4층  부산시 기장군 기장읍 동부리    376-1   jupiterkw@naver.com    28   8.0   
6      2층   부산시 기장군 기장읍 교리   354-13  ejskdms123@naver.com    30  10.0   
7      3층   부산시 기장군 기장읍 교리   350-10  ejskdms123@naver.com    32  12.0   
8      2층  부산시 기장군 기장읍 대라리   188-15  ejskdms123@naver.com    22   6.0   
9     13층  부산시 기장군 기장읍 대라리     91-6  ejskdms123@naver.com    42  15.0   
10     3층  부산시 기장군 기장읍 대라리    414-9  ejskdms123@naver.com    40  15.0   
11     4층  부산시 기장군 기장읍 대라리   129-11   sonygod80@naver.com    36  12.0   
12     4층  부산시 기장군 기장읍 대라리    269-9  cleanman07@nav